In [1]:
import math
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng
from IPython.display import Image
import simpy
from pathlib import Path

Chunk below sets the random seed for similation. I had a hard time finding a point where I could insert this block into main functions listed below. 

In [2]:
rg = default_rng(seed=4470)

Next chunk establishes class object for the coffee shop we are trying to replicate. Given more time I would have liked to have fine tuned the values given to mean_order_time and mean_fulfillment_time, and their corresponding standard deviations. 

In [3]:
class coffee_shop(object):
    def __init__(self, env, num_cashiers = 2, num_baristas = 2, mean_order_time = .4, sd_order_time = .05, mean_fulfillment_time = .75, sd_fulfillment_time = .10, rg = rg):
        
        self.env = env
        self.rg = rg
        
        self.cashiers = simpy.Resource(env, num_cashiers)
        self.baristas = simpy.Resource(env, num_baristas)
        
        self.mean_order_time = mean_order_time
        self.sd_order_time = sd_order_time
        
        self.mean_fulfillment_time = mean_fulfillment_time
        self.sd_fulfillment_time = sd_fulfillment_time
        
    def order_drinks(self, customer):
        yield self.env.timeout(self.rg.normal(self.mean_order_time, self.sd_order_time))
        
    def get_drinks(self, customer):
        yield self.env.timeout(self.rg.normal(self.mean_fulfillment_time, self.sd_fulfillment_time))

The get_coffee function runs for each 'customer' that passes through our similation. Their order quantity is randomly assigned using Poisson distribution, to remedy zeros I added 1 to each generated value. Cashier and Barista resources must address all order items before being released to next customer. 

In [4]:
def get_coffee(env, customer, Coffee_shop):
    arrival_time = env.now
    order_qty = rg.poisson(1) + 1
    
    with Coffee_shop.cashiers.request() as request:
        yield request
        reached_cashier = env.now
        for i in range(order_qty):
            yield env.process(Coffee_shop.order_drinks(customer))
            # print(f"Customer {customer + 1} order for item {i + 1} of {order_qty} is placed at time {env.now:.2f}")
    order_placed = env.now
    
    with Coffee_shop.baristas.request() as request:
        yield request
        reached_barista = env.now
        for i in range(order_qty):
            yield env.process(Coffee_shop.get_drinks(customer))
            # print(f"Customer {customer + 1} order for item {i + 1} of {order_qty} is fulfilled at time {env.now:.2f}")
    order_fulfilled = env.now
    
    total_drinks.append(order_qty + 1)
    wait_times.append(env.now - arrival_time)
    wait_time = env.now - arrival_time
    order_to_fulfillment_wait = order_fulfilled - order_placed
    
    timestamps = {'customer_id': customer + 1,
                  'arrival_time': arrival_time,
                  'reached_cashier': reached_cashier,
                  'order_qty': order_qty,
                  'order_placed': order_placed,
                  'reached_barista': reached_barista,
                  'order_fulfilled': order_fulfilled,
                  'order_processing_time': order_to_fulfillment_wait,
                  'total_wait': wait_time}
    
    timestamps_list.append(timestamps)
    

The run_coffee_shop function processes the similation using input parameters. Customer arrival times are established with user supplied mean arrival time and generated with a poisson distribution. 

In [5]:
def run_coffee_shop(env, num_cashiers, num_baristas, mean_arrival_time, stoptime, max_arrivals):
    Coffee_shop = coffee_shop(env, num_cashiers, num_baristas)
    
    for customer in range(3):
        env.process(get_coffee(env, customer, Coffee_shop))
    
    while env.now < stoptime and customer < max_arrivals:
        iat = rg.poisson(mean_arrival_time)
        
        yield env.timeout(iat)
        
        customer += 1
        
        env.process(get_coffee(env, customer, Coffee_shop))
        
    print(f"{customer} customers have arrived.")
    
    

Using get_user_input someone can run simulation multiple times and generate multiple csvs for various scenarios. 

In [6]:
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_baristas = input("Input # of baristas working: ")
    mean_arrival_time = input("Input customer mean arrival time:")
    stoptime = input("Input stoptime:")
    max_arrivals = input("Input max number of arrivals")
    
    params = [num_cashiers, num_baristas, mean_arrival_time, stoptime, max_arrivals]
    
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 cashier, 2 baristas, a 3 minutes average arrival time, 480 max stoptime and 600 max arrivals.",
        )
        params = [1, 2, 3, 480, 600]
    return params

The two chunks below provide average wait time and maximum wait time for summary stats at the conclusion of simulation. 

In [7]:
def get_average_wait_time(wait_times):
    average_wait = np.mean(wait_times)
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [8]:
def get_max_wait_time(wait_times):
    max_wait = np.max(wait_times)
    minutes, frac_minutes = divmod(max_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

This chuck initiates the user input prompts and kick starts the similation. 

In [9]:
# generate empty lists for current simulation
total_drinks = []
wait_times = []
timestamps_list = []
def main():
    # gather user input and attribute values to appropriate variables
    num_cashiers, num_baristas, mean_arrival_time, stoptime, max_arrivals = get_user_input()
    
    print("Running simulation...")

    # Run the simulation
    env = simpy.Environment()
    env.process(run_coffee_shop(env, num_cashiers, num_baristas, mean_arrival_time, stoptime, max_arrivals))
    env.run()
    
    # Create data frame and update values to identify parameters. Then save as csv to output folder. 
    df = pd.DataFrame(timestamps_list)
    df['num_cashiers'] = num_cashiers
    df['num_baristas'] = num_baristas
    df['mean_arrival_time'] = mean_arrival_time
    df_name = f"cafe_sim_cashiers{num_cashiers}_baristas{num_baristas}_MAT{mean_arrival_time}"
    output_path = f'../output/{df_name}'
    df.to_csv(output_path, index=False)
    
    # Give some high level results
    tot_drinks = sum(total_drinks)
    mins, secs = get_average_wait_time(wait_times)
    mx_mins, mx_secs = get_max_wait_time(wait_times)
    print(
        f"\nCoffee shop made {tot_drinks} drinks.",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
        f"\nThe longest wait time was {mx_mins} minutes and {mx_secs} seconds.",
    )


if __name__ == "__main__":
    main()

Input # of cashiers working:  1
Input # of baristas working:  3
Input customer mean arrival time: 4
Input stoptime: 250
Input max number of arrivals 500


Running simulation...
67 customers have arrived.

Coffee shop made 205 drinks. 
The average wait time is 2 minutes and 25 seconds. 
The longest wait time was 5 minutes and 3 seconds.
